### Setup


R code for ordinal regression


In [ ]:
library(readr)
library(MASS)
library(ordinal)
library(brant)
library(broom)
library(generalhoslem)
library(boot)

Loading required package: reshape



In [ ]:
if (file.exists("data/debates_augmented.csv")) {
    mydf <- read_csv("data/debates_augmented.csv")
} else {
    cat("File not found: data/debates_augmented.csv. Please run `python process_data.py` first.")
}

In [3]:
mydf$sideAgreementPostTreatment = ordered(mydf$sideAgreementPostTreatment)

mydf$sideAgreementPreTreatment = factor(mydf$sideAgreementPreTreatment, levels=c(1, 2, 3, 4, 5))
backward.diff = matrix(c(-4/5, 1/5, 1/5, 1/5, 1/5, -3/5, -3/5, 2/5, 2/5, 2/5, -2/5, 
                         -2/5, -2/5, 3/5, 3/5, -1/5, -1/5, -1/5, -1/5, 4/5), ncol = 4)
contrasts(mydf$sideAgreementPreTreatment) = backward.diff

mydf<- mydf[mydf$treatmentType %in% c("Human-Human", "Human-AI", "Human-Human, personalized", "Human-AI, personalized"), ]
mydf$treatmentType = factor(mydf$treatmentType, levels=c("Human-Human", "Human-AI", "Human-Human, personalized", "Human-AI, personalized"))

In [4]:
mydf$gender = factor(mydf$gender, levels=c("male", "female", "other"))
mydf$age = factor(mydf$age, levels=c("18-24", "25-34", "35-44", "45-54", "55-64", "65+"))
mydf$ethnicity = factor(mydf$ethnicity, levels=c("white", "black", "asian", "latino", "mixed", "other"))
mydf$education = factor(mydf$education, levels=c("high-school", "no-degree", "vocational", "bachelor", "master", "phd"))
mydf$employmentStatus = factor(mydf$employmentStatus, levels=c("employed-wages", "self-employed", "unemployed", "student", "retired", "other"))
mydf$politicalAffiliation = factor(mydf$politicalAffiliation, levels=c("democrat", "republican", "independent", "other"))

mydf$topicCluster = factor(mydf$topicCluster, levels=c(1, 2, 3))
mydf$topicClusterNew = factor(mydf$topicClusterNew, levels=c(1,  2, 3))
mydf$strengthPreTreatment = factor(mydf$strengthPreTreatment, levels=c(0, 1, 2))

# normalize to range [0, 1]
mydf$overallOpponent_Analytic = mydf$overallOpponent_Analytic / 100
mydf$overallOpponent_Clout = mydf$overallOpponent_Clout / 100
mydf$overallOpponent_Authentic = mydf$overallOpponent_Authentic / 100
mydf$overallOpponent_Tone = mydf$overallOpponent_Tone / 100
mydf$overallOpponentFlesch = mydf$overallOpponentFlesch / 100

### Brant test


In [5]:
# Testing the parallel lines assumption
model <- polr(formula = sideAgreementPostTreatment ~ treatmentType + sideAgreementPreTreatment, data=mydf, Hess=TRUE)
# summary(model)
brant(model)

---------------------------------------------------------------------------- 
Test for					X2	df	probability 
---------------------------------------------------------------------------- 
Omnibus						128.32	21	0
treatmentTypeHuman-AI				5.89	3	0.12
treatmentTypeHuman-Human, personalized	10.03	3	0.02
treatmentTypeHuman-AI, personalized		3.35	3	0.34
sideAgreementPreTreatment1			36.71	3	0
sideAgreementPreTreatment2			9.3	3	0.03
sideAgreementPreTreatment3			22.13	3	0
sideAgreementPreTreatment4			32.08	3	0
---------------------------------------------------------------------------- 

H0: Parallel Regression Assumption holds


Warning message in brant(model):
"6 combinations in table(dv,ivs) do not occur. Because of that, the test results might be invalid."


### Key result


In [6]:
# Partial Proportional Odds. Now only treatmentType uses the parallel lines assumption
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType, nominal= ~ sideAgreementPreTreatment, data=mydf, link="logit")
write.csv(tidy(model), "data/analysis_paper/model.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)


dof: 727 


formula: sideAgreementPostTreatment ~ treatmentType
nominal: ~sideAgreementPreTreatment
data:    mydf

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  750  -957.53 1961.06 8(0)  1.26e-08 3.3e+02

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)   
treatmentTypeHuman-AI                    0.1978     0.1914   1.034  0.30118   
treatmentTypeHuman-Human, personalized  -0.1709     0.1929  -0.886  0.37547   
treatmentTypeHuman-AI, personalized      0.5945     0.1856   3.204  0.00136 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                 -1.7102     0.1733  -9.868
2|3.(Intercept)                 -0.4400     0.1246  -3.532
3|4.(Intercept)                  1.0926     0.1298   8.420
4|5.(Intercept)                  2.3581     0.1659  14.213
1|2.sideAgreementPreTreatment1  -1.8869     0.2530  -7.45

In [7]:
lipsitz.test(model, 20)

Warning message:
"contrasts dropped from factor sideAgreementPreTreatment"



	Lipsitz goodness of fit test for ordinal response models

data:  formula:  sideAgreementPostTreatment ~ treatmentType
LR statistic = 10.87, df = 19, p-value = 0.9281


In [8]:
pulkrob.deviance(model, c("treatmentType"))

Warning message:
"contrasts dropped from factor sideAgreementPreTreatment"



	Pulkstenis-Robinson deviance test

data:  formula:  sideAgreementPostTreatment ~ treatmentType
Deviance-squared = 35.861, df = 26, p-value = 0.09426


In [9]:
pulkrob.chisq(model, c("treatmentType"))

Warning message:
"contrasts dropped from factor sideAgreementPreTreatment"



	Pulkstenis-Robinson chi-squared test

data:  formula:  sideAgreementPostTreatment ~ treatmentType
X-squared = 34.189, df = 26, p-value = 0.1303


In [10]:
predprob <- data.frame(treatmentType = mydf$treatmentType, sideAgreementPreTreatment = mydf$sideAgreementPreTreatment)
fv <- predict(model, newdata = predprob, type = "prob")$fit
logitgof(mydf$sideAgreementPostTreatment, fv, ord = TRUE)

Warning message:
"contrasts dropped from factor sideAgreementPreTreatment"



	Hosmer and Lemeshow test (ordinal model)

data:  mydf$sideAgreementPostTreatment, fv
X-squared = 18.623, df = 35, p-value = 0.9894


### Pre-registration analysis


In [11]:
mydf_prereg <- mydf[mydf$treatmentType %in% c("Human-Human", "Human-AI", "Human-AI, personalized"), ]

# Create 9 treatments, combining the 3 treatmentTypes with the 3 levels of topicClusterNew
mydf_prereg$treatmentNew = paste(mydf_prereg$treatmentType, mydf_prereg$topicClusterNew, sep = "_")
mydf_prereg$treatmentNew = factor(mydf_prereg$treatmentNew, levels=c("Human-Human_1", "Human-Human_2", "Human-Human_3", "Human-AI_1", "Human-AI_2", "Human-AI_3", "Human-AI, personalized_1", "Human-AI, personalized_2", "Human-AI, personalized_3"))

In [12]:
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentNew, nominal= ~ sideAgreementPreTreatment, data=mydf_prereg, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_preregistration1.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 572 


formula: sideAgreementPostTreatment ~ treatmentNew
nominal: ~sideAgreementPreTreatment
data:    mydf_prereg

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  600  -763.52 1583.05 6(0)  2.21e-07 2.6e+02

Coefficients:
                                     Estimate Std. Error z value Pr(>|z|)  
treatmentNewHuman-Human_2            -0.01599    0.26618  -0.060   0.9521  
treatmentNewHuman-Human_3             0.14919    0.27520   0.542   0.5877  
treatmentNewHuman-AI_1                0.22916    0.32745   0.700   0.4840  
treatmentNewHuman-AI_2                0.35997    0.32909   1.094   0.2740  
treatmentNewHuman-AI_3                0.10720    0.34719   0.309   0.7575  
treatmentNewHuman-AI, personalized_1  0.60765    0.31706   1.917   0.0553 .
treatmentNewHuman-AI, personalized_2  0.69559    0.32679   2.129   0.0333 *
treatmentNewHuman-AI, personalized_3  0.67101    0.32773   2.047   0.0406 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Th

In [13]:
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType * topicClusterNew, nominal= ~ sideAgreementPreTreatment, data=mydf_prereg, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_preregistration2.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 572 


formula: sideAgreementPostTreatment ~ treatmentType * topicClusterNew
nominal: ~sideAgreementPreTreatment
data:    mydf_prereg

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  600  -763.52 1583.05 6(0)  2.21e-07 2.6e+02

Coefficients:
                                                     Estimate Std. Error
treatmentTypeHuman-AI                                 0.22916    0.32745
treatmentTypeHuman-AI, personalized                   0.60765    0.31706
topicClusterNew2                                     -0.01599    0.26618
topicClusterNew3                                      0.14919    0.27520
treatmentTypeHuman-AI:topicClusterNew2                0.14680    0.46487
treatmentTypeHuman-AI, personalized:topicClusterNew2  0.10393    0.45305
treatmentTypeHuman-AI:topicClusterNew3               -0.27115    0.47940
treatmentTypeHuman-AI, personalized:topicClusterNew3 -0.08583    0.45535
                                                     z value Pr(>|z|)  
treatm

### Topic Analysis


In [14]:
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType, nominal= ~ sideAgreementPreTreatment, data=mydf[mydf$topicClusterNew == 1,], link="logit")
write.csv(tidy(model), "data/analysis_paper/model_topic_cluster1.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 227 


formula: sideAgreementPostTreatment ~ treatmentType
nominal: ~sideAgreementPreTreatment
data:    mydf[mydf$topicClusterNew == 1, ]

 link  threshold nobs logLik  AIC    niter max.grad cond.H 
 logit flexible  250  -315.31 676.62 6(0)  1.83e-09 3.0e+02

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)  
treatmentTypeHuman-AI                   0.15613    0.33232   0.470   0.6385  
treatmentTypeHuman-Human, personalized  0.08543    0.33787   0.253   0.8004  
treatmentTypeHuman-AI, personalized     0.67868    0.32388   2.095   0.0361 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                 -1.8262     0.3037  -6.014
2|3.(Intercept)                 -0.3273     0.2170  -1.508
3|4.(Intercept)                  1.3893     0.2427   5.725
4|5.(Intercept)                  2.6345     0.3323   7.929
1|2.sideAgreementPreTreatment1  -2

In [15]:
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType, nominal= ~ sideAgreementPreTreatment, data=mydf[mydf$topicClusterNew == 2,], link="logit")
write.csv(tidy(model), "data/analysis_paper/model_topic_cluster2.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 227 


formula: sideAgreementPostTreatment ~ treatmentType
nominal: ~sideAgreementPreTreatment
data:    mydf[mydf$topicClusterNew == 2, ]

 link  threshold nobs logLik  AIC    niter max.grad cond.H 
 logit flexible  250  -327.69 701.39 6(1)  3.30e-07 3.0e+02

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)  
treatmentTypeHuman-AI                  0.318197   0.326377   0.975   0.3296  
treatmentTypeHuman-Human, personalized 0.006303   0.338331   0.019   0.9851  
treatmentTypeHuman-AI, personalized    0.649980   0.324669   2.002   0.0453 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                -1.64754    0.29769  -5.534
2|3.(Intercept)                -0.34253    0.21705  -1.578
3|4.(Intercept)                 1.05421    0.22415   4.703
4|5.(Intercept)                 2.46254    0.30046   8.196
1|2.sideAgreementPreTreatment1 -1.

In [16]:
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType, nominal= ~ sideAgreementPreTreatment, data=mydf[mydf$topicClusterNew == 3,], link="logit")
write.csv(tidy(model), "data/analysis_paper/model_topic_cluster3.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 227 


formula: sideAgreementPostTreatment ~ treatmentType
nominal: ~sideAgreementPreTreatment
data:    mydf[mydf$topicClusterNew == 3, ]

 link  threshold nobs logLik  AIC    niter max.grad cond.H 
 logit flexible  250  -296.58 639.17 7(2)  1.84e-09 6.6e+02

Coefficients:
                                        Estimate Std. Error z value Pr(>|z|)  
treatmentTypeHuman-AI                   0.005693   0.349796   0.016   0.9870  
treatmentTypeHuman-Human, personalized -0.614073   0.347194  -1.769   0.0769 .
treatmentTypeHuman-AI, personalized     0.495813   0.334195   1.484   0.1379  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                -1.65745    0.31094  -5.330
2|3.(Intercept)                -0.62468    0.23069  -2.708
3|4.(Intercept)                 0.89559    0.23918   3.744
4|5.(Intercept)                 2.20000    0.30270   7.268
1|2.sideAgreementPreTreatment1

### Linear regression


In [17]:
model <- lm(formula = delta ~ treatmentType, data=mydf)
write.csv(tidy(model), "data/analysis_paper/model_linear.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 746 



Call:
lm(formula = delta ~ treatmentType, data = mydf)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.9800 -0.7767  0.1600  0.2233  4.2233 

Coefficients:
                                       Estimate Std. Error t value Pr(>|t|)   
(Intercept)                            -0.22333    0.07116  -3.139  0.00176 **
treatmentTypeHuman-AI                   0.20333    0.12325   1.650  0.09941 . 
treatmentTypeHuman-Human, personalized  0.06333    0.12325   0.514  0.60750   
treatmentTypeHuman-AI, personalized     0.36333    0.12325   2.948  0.00330 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.232 on 746 degrees of freedom
Multiple R-squared:  0.0128,	Adjusted R-squared:  0.008834 
F-statistic: 3.225 on 3 and 746 DF,  p-value: 0.02209


### Demographics


In [18]:
# Demographics
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + gender + age + ethnicity + education + employmentStatus + politicalAffiliation, nominal= ~ sideAgreementPreTreatment, data=mydf, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_demographics.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 701 


formula: 
sideAgreementPostTreatment ~ treatmentType + gender + age + ethnicity + education + employmentStatus + politicalAffiliation
nominal: ~sideAgreementPreTreatment
data:    mydf

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  749  -946.63 1989.26 8(0)  6.56e-08 3.3e+02

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)   
treatmentTypeHuman-AI                   0.20545    0.19647   1.046  0.29570   
treatmentTypeHuman-Human, personalized -0.21093    0.19914  -1.059  0.28951   
treatmentTypeHuman-AI, personalized     0.55566    0.19059   2.916  0.00355 **
genderfemale                            0.13192    0.14667   0.899  0.36840   
genderother                             0.35222    0.43958   0.801  0.42298   
age25-34                                0.15184    0.27466   0.553  0.58039   
age35-44                               -0.03586    0.29363  -0.122  0.90280   
age45-54                               -0.0975

### Mediation


In [19]:
# LIWC
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + overallOpponent_i + overallOpponent_we + overallOpponent_you + overallOpponent_emo_pos + overallOpponent_emo_neg + overallOpponent_Analytic + overallOpponent_Clout + overallOpponent_Authentic + overallOpponent_Tone + log(overallOpponent_WC) + overallOpponentFlesch, nominal= ~ sideAgreementPreTreatment, data=mydf, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_liwc.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 716 


formula: 
sideAgreementPostTreatment ~ treatmentType + overallOpponent_i + overallOpponent_we + overallOpponent_you + overallOpponent_emo_pos + overallOpponent_emo_neg + overallOpponent_Analytic + overallOpponent_Clout + overallOpponent_Authentic + overallOpponent_Tone + log(overallOpponent_WC) + overallOpponentFlesch
nominal: ~sideAgreementPreTreatment
data:    mydf

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  750  -954.61 1977.22 8(0)  5.42e-12 4.1e+04

Coefficients:
                                        Estimate Std. Error z value Pr(>|z|)  
treatmentTypeHuman-AI                  -0.039922   0.303860  -0.131    0.895  
treatmentTypeHuman-Human, personalized -0.176725   0.198541  -0.890    0.373  
treatmentTypeHuman-AI, personalized     0.403674   0.290995   1.387    0.165  
overallOpponent_i                      -0.124274   0.070034  -1.774    0.076 .
overallOpponent_we                      0.006493   0.057599   0.113    0.910  
overallOpponent_yo

In [20]:
# Social dimensions
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + argumentOpponent_knowledge_mean + argumentOpponent_similarity_mean + argumentOpponent_trust_mean + argumentOpponent_power_mean + argumentOpponent_support_mean + argumentOpponent_identity_mean + argumentOpponent_conflict_mean + argumentOpponent_status_mean + argumentOpponent_fun_mean, nominal= ~ sideAgreementPreTreatment, data=mydf, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_dimensions.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 718 


formula: 
sideAgreementPostTreatment ~ treatmentType + argumentOpponent_knowledge_mean + argumentOpponent_similarity_mean + argumentOpponent_trust_mean + argumentOpponent_power_mean + argumentOpponent_support_mean + argumentOpponent_identity_mean + argumentOpponent_conflict_mean + argumentOpponent_status_mean + argumentOpponent_fun_mean
nominal: ~sideAgreementPreTreatment
data:    mydf

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  750  -952.30 1968.60 8(0)  4.10e-09 3.0e+03

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)    
treatmentTypeHuman-AI                   0.41950    0.24987   1.679 0.093187 .  
treatmentTypeHuman-Human, personalized -0.16760    0.19462  -0.861 0.389133    
treatmentTypeHuman-AI, personalized     0.80750    0.24282   3.326 0.000883 ***
argumentOpponent_knowledge_mean        -1.08148    0.89522  -1.208 0.227021    
argumentOpponent_similarity_mean        0.59888    0.43753   1.369 0.1710

In [21]:
# LIWC, with KHB

model_overallOpponent_i = lm(formula = overallOpponent_i ~ treatmentType, data=mydf)
model_overallOpponent_we = lm(formula = overallOpponent_we ~ treatmentType, data=mydf)
model_overallOpponent_you = lm(formula = overallOpponent_you ~ treatmentType, data=mydf)
model_overallOpponent_emo_pos = lm(formula = overallOpponent_emo_pos ~ treatmentType, data=mydf)
model_overallOpponent_emo_neg = lm(formula = overallOpponent_emo_neg ~ treatmentType, data=mydf)
model_overallOpponent_Analytic = lm(formula = overallOpponent_Analytic ~ treatmentType, data=mydf)
model_overallOpponent_Clout = lm(formula = overallOpponent_Clout ~ treatmentType, data=mydf)
model_overallOpponent_Authentic = lm(formula = overallOpponent_Authentic ~ treatmentType, data=mydf)
model_overallOpponent_Tone = lm(formula = overallOpponent_Tone ~ treatmentType, data=mydf)
model_overallOpponent_WC = lm(formula = log(overallOpponent_WC) ~ treatmentType, data=mydf)
model_overallOpponentFlesch = lm(formula = overallOpponentFlesch ~ treatmentType, data=mydf)

# Compute residuals
newdf <- mydf
newdf$overallOpponent_i_residuals = residuals(model_overallOpponent_i)
newdf$overallOpponent_we_residuals = residuals(model_overallOpponent_we)
newdf$overallOpponent_you_residuals = residuals(model_overallOpponent_you)
newdf$overallOpponent_emo_pos_residuals = residuals(model_overallOpponent_emo_pos)  
newdf$overallOpponent_emo_neg_residuals = residuals(model_overallOpponent_emo_neg)
newdf$overallOpponent_Analytic_residuals = residuals(model_overallOpponent_Analytic)
newdf$overallOpponent_Clout_residuals = residuals(model_overallOpponent_Clout)
newdf$overallOpponent_Authentic_residuals = residuals(model_overallOpponent_Authentic)
newdf$overallOpponent_Tone_residuals = residuals(model_overallOpponent_Tone)
newdf$overallOpponent_WC_residuals = residuals(model_overallOpponent_WC)
newdf$overallOpponentFlesch_residuals = residuals(model_overallOpponentFlesch)

model_full <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + overallOpponent_i + overallOpponent_we + overallOpponent_you + overallOpponent_emo_pos + overallOpponent_emo_neg + overallOpponent_Analytic + overallOpponent_Clout + overallOpponent_Authentic + overallOpponent_Tone + log(overallOpponent_WC) + overallOpponentFlesch, nominal= ~ sideAgreementPreTreatment, data=mydf, link="logit")
model_residual <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + overallOpponent_i_residuals + overallOpponent_we_residuals + overallOpponent_you_residuals + overallOpponent_emo_pos_residuals + overallOpponent_emo_neg_residuals + overallOpponent_Analytic_residuals + overallOpponent_Clout_residuals + overallOpponent_Authentic_residuals + overallOpponent_Tone_residuals + overallOpponent_WC_residuals + overallOpponentFlesch_residuals, nominal= ~ sideAgreementPreTreatment, data=newdf, link="logit")

# Percentage change attributable to mediation
1 - model_full$coefficients["treatmentTypeHuman-AI"] / model_residual$coefficients["treatmentTypeHuman-AI"]
1 - model_full$coefficients["treatmentTypeHuman-Human, personalized"] / model_residual$coefficients["treatmentTypeHuman-Human, personalized"]
1 - model_full$coefficients["treatmentTypeHuman-AI, personalized"] / model_residual$coefficients["treatmentTypeHuman-AI, personalized"]

treatmentTypeHuman-AI 
              1.20062

treatmentTypeHuman-Human, personalized 
                          -0.001059197

treatmentTypeHuman-AI, personalized 
                          0.3282583

In [22]:
# Bootstrap confidence intervals

ba <- function(data, indices)
{
    bdf = data[indices, ]
    
    model_overallOpponent_i = lm(formula = overallOpponent_i ~ treatmentType, data=bdf)
    model_overallOpponent_we = lm(formula = overallOpponent_we ~ treatmentType, data=bdf)
    model_overallOpponent_you = lm(formula = overallOpponent_you ~ treatmentType, data=bdf)
    model_overallOpponent_emo_pos = lm(formula = overallOpponent_emo_pos ~ treatmentType, data=bdf)
    model_overallOpponent_emo_neg = lm(formula = overallOpponent_emo_neg ~ treatmentType, data=bdf)
    model_overallOpponent_Analytic = lm(formula = overallOpponent_Analytic ~ treatmentType, data=bdf)
    model_overallOpponent_Clout = lm(formula = overallOpponent_Clout ~ treatmentType, data=bdf)
    model_overallOpponent_Authentic = lm(formula = overallOpponent_Authentic ~ treatmentType, data=bdf)
    model_overallOpponent_Tone = lm(formula = overallOpponent_Tone ~ treatmentType, data=bdf)
    model_overallOpponent_WC = lm(formula = log(overallOpponent_WC) ~ treatmentType, data=bdf)
    model_overallOpponentFlesch = lm(formula = overallOpponentFlesch ~ treatmentType, data=bdf)

    # Compute residuals
    newdf <- bdf
    newdf$overallOpponent_i_residuals = residuals(model_overallOpponent_i)
    newdf$overallOpponent_we_residuals = residuals(model_overallOpponent_we)
    newdf$overallOpponent_you_residuals = residuals(model_overallOpponent_you)
    newdf$overallOpponent_emo_pos_residuals = residuals(model_overallOpponent_emo_pos)
    newdf$overallOpponent_emo_neg_residuals = residuals(model_overallOpponent_emo_neg)
    newdf$overallOpponent_Analytic_residuals = residuals(model_overallOpponent_Analytic)
    newdf$overallOpponent_Clout_residuals = residuals(model_overallOpponent_Clout)
    newdf$overallOpponent_Authentic_residuals = residuals(model_overallOpponent_Authentic)
    newdf$overallOpponent_Tone_residuals = residuals(model_overallOpponent_Tone)
    newdf$overallOpponent_WC_residuals = residuals(model_overallOpponent_WC)
    newdf$overallOpponentFlesch_residuals = residuals(model_overallOpponentFlesch)

    model_full <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + overallOpponent_i + overallOpponent_we + overallOpponent_you + overallOpponent_emo_pos + overallOpponent_emo_neg + overallOpponent_Analytic + overallOpponent_Clout + overallOpponent_Authentic + overallOpponent_Tone + log(overallOpponent_WC) + overallOpponentFlesch, nominal= ~ sideAgreementPreTreatment, data=bdf, link="logit")
    model_residual <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + overallOpponent_i_residuals + overallOpponent_we_residuals + overallOpponent_you_residuals + overallOpponent_emo_pos_residuals + overallOpponent_emo_neg_residuals + overallOpponent_Analytic_residuals + overallOpponent_Clout_residuals + overallOpponent_Authentic_residuals + overallOpponent_Tone_residuals + overallOpponent_WC_residuals + overallOpponentFlesch_residuals, nominal= ~ sideAgreementPreTreatment, data=newdf, link="logit")

    if (model_full$cond.H > 1e8)
    {
        return(c(NA, NA, NA))
    }

    result = c(1 - model_full$coefficients["treatmentTypeHuman-AI"] / model_residual$coefficients["treatmentTypeHuman-AI"],
               1 - model_full$coefficients["treatmentTypeHuman-Human, personalized"] / model_residual$coefficients["treatmentTypeHuman-Human, personalized"],
               1 - model_full$coefficients["treatmentTypeHuman-AI, personalized"] / model_residual$coefficients["treatmentTypeHuman-AI, personalized"])

    return(result)
}

set.seed(42)
boot_results <- boot(data=mydf, statistic=ba, R=1000)

Warning message:
"(1) Hessian is numerically singular: parameters are not uniquely determined 
In addition: Absolute convergence criterion was met, but relative criterion was not met"
Warning message:
"(1) Hessian is numerically singular: parameters are not uniquely determined 
In addition: Absolute convergence criterion was met, but relative criterion was not met"
Warning message:
"(1) Hessian is numerically singular: parameters are not uniquely determined 
In addition: Absolute convergence criterion was met, but relative criterion was not met"
Warning message:
"(1) Hessian is numerically singular: parameters are not uniquely determined 
In addition: Absolute convergence criterion was met, but relative criterion was not met"
Warning message:
"(-3) not all thresholds are increasing: fit is invalid 
In addition: Absolute convergence criterion was met, but relative criterion was not met"
Warning message:
"(-3) not all thresholds are increasing: fit is invalid 
In addition: Absolute conve

In [23]:
boot_results


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = mydf, statistic = ba, R = 1000)


Bootstrap Statistics :
        original     bias    std. error
t1*  1.200620184 -0.7076269   30.260289
t2* -0.001059197  0.2926603    5.696961
t3*  0.328258319  0.2546496    6.194360

In [24]:
boot.ci(boot_results, 0.95, index=3)

Warning message in boot.ci(boot_results, 0.95, index = 3):
"bootstrap variances needed for studentized intervals"


BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 940 bootstrap replicates

CALL : 
boot.ci(boot.out = boot_results, conf = 0.95, index = 3)

Intervals : 
Level      Normal              Basic         
95%   (-12.0671,  12.2143 )   ( -1.0098,   1.1276 )  

Level     Percentile            BCa          
95%   (-0.4711,  1.6663 )   (-0.4849,  1.6127 )  
Calculations and Intervals on Original Scale

### Opinion fluidity


In [25]:
model <- glm(formula = changedAgreement ~ treatmentType + topicPrior + topicClusterNew + topicKnowledge + topicDebatableness, data=mydf, family="binomial")
write.csv(tidy(model), "data/analysis_paper/opinion_fluidity.csv", row.names = FALSE)
cat("nobs: ", nobs(model), "\n")
cat("dof:", model$df.residual, "\n")
summary(model)

nobs:  750 
dof: 741 



Call:
glm(formula = changedAgreement ~ treatmentType + topicPrior + 
    topicClusterNew + topicKnowledge + topicDebatableness, family = "binomial", 
    data = mydf)

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)   
(Intercept)                             2.57360    1.80859   1.423  0.15474   
treatmentTypeHuman-AI                   0.01203    0.20278   0.059  0.95270   
treatmentTypeHuman-Human, personalized  0.01294    0.20286   0.064  0.94913   
treatmentTypeHuman-AI, personalized     0.39734    0.20513   1.937  0.05274 . 
topicPrior                             -0.14014    0.06477  -2.164  0.03050 * 
topicClusterNew2                        0.21393    0.18576   1.152  0.24947   
topicClusterNew3                        0.01641    0.18826   0.087  0.93055   
topicKnowledge                         -1.31233    0.40633  -3.230  0.00124 **
topicDebatableness                      0.79543    0.42315   1.880  0.06014 . 
---
Signif. codes:  0 '***'

### Perceived opponent


In [26]:
model <- glm(formula = perceivedOpponentBinary ~ treatmentType + overallOpponent_i + overallOpponent_we + overallOpponent_you + overallOpponent_emo_pos + overallOpponent_emo_neg + overallOpponent_Analytic + overallOpponent_Clout + overallOpponent_Authentic + overallOpponent_Tone + log(overallOpponent_WC) + overallOpponentFlesch , data=mydf, family="binomial")
write.csv(tidy(model), "data/analysis_paper/perceived_opponent.csv", row.names = FALSE)
cat("nobs: ", nobs(model), "\n")
cat("dof:", model$df.residual, "\n")
summary(model)

nobs:  750 
dof: 735 



Call:
glm(formula = perceivedOpponentBinary ~ treatmentType + overallOpponent_i + 
    overallOpponent_we + overallOpponent_you + overallOpponent_emo_pos + 
    overallOpponent_emo_neg + overallOpponent_Analytic + overallOpponent_Clout + 
    overallOpponent_Authentic + overallOpponent_Tone + log(overallOpponent_WC) + 
    overallOpponentFlesch, family = "binomial", data = mydf)

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)  
(Intercept)                            -1.22245    1.33643  -0.915   0.3603  
treatmentTypeHuman-AI                   0.59260    0.34228   1.731   0.0834 .
treatmentTypeHuman-Human, personalized -0.25552    0.20872  -1.224   0.2209  
treatmentTypeHuman-AI, personalized     0.44300    0.33140   1.337   0.1813  
overallOpponent_i                      -0.11675    0.07829  -1.491   0.1359  
overallOpponent_we                     -0.02633    0.06451  -0.408   0.6831  
overallOpponent_you                     0.07771    0.117

In [27]:
# Including perceivedOpponent
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + perceivedOpponentBinary, nominal= ~ sideAgreementPreTreatment, data=mydf, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_perceivedOpponent.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 726 


formula: sideAgreementPostTreatment ~ treatmentType + perceivedOpponentBinary
nominal: ~sideAgreementPreTreatment
data:    mydf

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  750  -955.18 1958.36 8(0)  3.23e-10 3.3e+02

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)   
treatmentTypeHuman-AI                    0.1199     0.1947   0.616  0.53812   
treatmentTypeHuman-Human, personalized  -0.1387     0.1937  -0.716  0.47384   
treatmentTypeHuman-AI, personalized      0.5319     0.1879   2.831  0.00464 **
perceivedOpponentBinary                  0.3178     0.1469   2.164  0.03046 * 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                 -1.5507     0.1879  -8.251
2|3.(Intercept)                 -0.2738     0.1462  -1.873
3|4.(Intercept)                  1.2665     0.1532   8.264
4|5.(Interc

### Opponent agreement pre-treatment


In [28]:
# Only keep datapoints where the the pre-treatment agreements are consistent with the assigned side
mydf_restricted = mydf[((mydf$treatmentType %in% c("Human-Human", "Human-Human, personalized") & 
                            (mydf$sideAgreementPreTreatment %in% c(1, 2) & mydf$opponentSideAgreementPreTreatment %in% c(1, 2))) 
                        | (mydf$treatmentType %in% c("Human-AI", "Human-AI, personalized") & 
                            mydf$sideAgreementPreTreatment %in% c(1, 2))),]

In [29]:
# print how many datapoints per treatmentType
table(mydf_restricted$treatmentType)


              Human-Human                  Human-AI Human-Human, personalized 
                       40                        69                        18 
   Human-AI, personalized 
                       62 

In [30]:
# Key analysis, restricted to people with pre-treatment scores consistent with the assigned side
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType, nominal= ~ sideAgreementPreTreatment, data=mydf_restricted, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_restricted.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

Warning message:
"contrasts dropped from factor sideAgreementPreTreatment due to missing levels"
Warning message:
"contrasts dropped from factor sideAgreementPreTreatment due to missing levels"


dof: 178 


formula: sideAgreementPostTreatment ~ treatmentType
nominal: ~sideAgreementPreTreatment
data:    mydf_restricted

 link  threshold nobs logLik  AIC    niter max.grad cond.H 
 logit flexible  189  -238.98 499.96 6(0)  1.62e-13 1.3e+02

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)  
treatmentTypeHuman-AI                   -0.2887     0.3814  -0.757    0.449  
treatmentTypeHuman-Human, personalized  -0.4789     0.5553  -0.862    0.388  
treatmentTypeHuman-AI, personalized      0.8011     0.3779   2.120    0.034 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                  0.5408     0.3497   1.547
2|3.(Intercept)                  1.3426     0.3713   3.616
3|4.(Intercept)                  2.3830     0.4468   5.333
4|5.(Intercept)                  3.5645     0.6503   5.481
1|2.sideAgreementPreTreatment2  -1.8445     0.3398  

In [31]:
mydf_human = mydf[mydf$treatmentType %in% c("Human-Human", "Human-Human, personalized"), ]
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + opponentSideAgreementPreTreatment, nominal= ~ sideAgreementPreTreatment, data=mydf_human, link="logit")
cat("dof:", model$df, "\n")
summary(model)

dof: 428 


formula: 
sideAgreementPostTreatment ~ treatmentType + opponentSideAgreementPreTreatment
nominal: ~sideAgreementPreTreatment
data:    mydf_human

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  450  -566.26 1176.52 7(1)  9.89e-13 1.7e+03

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)
treatmentTypeHuman-Human, personalized -0.17823    0.19422  -0.918    0.359
opponentSideAgreementPreTreatment       0.07747    0.06668   1.162    0.245

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                 -1.5033     0.2795  -5.379
2|3.(Intercept)                 -0.1510     0.2392  -0.632
3|4.(Intercept)                  1.2455     0.2477   5.028
4|5.(Intercept)                  2.4654     0.2803   8.795
1|2.sideAgreementPreTreatment1  -2.0473     0.3321  -6.166
2|3.sideAgreementPreTreatment1  -0.3793     0.3365  -1.127
3|4.sideAgreementPreTreatment1   0.4599     0.4829 

In [32]:
mydf_opponentagreement = mydf

# opponentSideAgreementPreTreatment should be 1 if the treatmentType is AI, and kept as it is otherwise
mydf_opponentagreement = transform(mydf_opponentagreement, opponentSideAgreementPreTreatment = ifelse(treatmentType %in% c("Human-AI", "Human-AI, personalized"), 3, opponentSideAgreementPreTreatment))

# # Including opponentSideAgreementPreTreatment
model <- ordinal::clm(formula = sideAgreementPostTreatment ~ treatmentType + opponentSideAgreementPreTreatment, nominal= ~ sideAgreementPreTreatment, data=mydf_opponentagreement, link="logit")
write.csv(tidy(model), "data/analysis_paper/model_opponentAgreement.csv", row.names = FALSE)
cat("dof:", model$df, "\n")
summary(model)

dof: 726 


formula: 
sideAgreementPostTreatment ~ treatmentType + opponentSideAgreementPreTreatment
nominal: ~sideAgreementPreTreatment
data:    mydf_opponentagreement

 link  threshold nobs logLik  AIC     niter max.grad cond.H 
 logit flexible  750  -956.77 1961.54 8(0)  2.90e-10 1.8e+03

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)   
treatmentTypeHuman-AI                   0.20214    0.19157   1.055   0.2914   
treatmentTypeHuman-Human, personalized -0.17069    0.19271  -0.886   0.3758   
treatmentTypeHuman-AI, personalized     0.59752    0.18578   3.216   0.0013 **
opponentSideAgreementPreTreatment       0.08116    0.06598   1.230   0.2187   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Threshold coefficients:
                               Estimate Std. Error z value
1|2.(Intercept)                 -1.4686     0.2617  -5.613
2|3.(Intercept)                 -0.1956     0.2345  -0.834
3|4.(Intercept)                  1.3384  